# API Scrapping Notebook

__by Gregory Rankel__

Within our notebook, we are looking to gather information on various subreddits on both football and american football and who are fans of both. To do that, we will tap in the API's of reddit and pull both information from the football reddit and NFL reddit as a means of gathering data. Within our function used to pull the data, we will also be doing a little bit of cleaning by dropping unnecessary data as well as dropping any duplicates that will be in there. We will then concatenate the two reddits into a Data Frame that will then be saved as a csv and used to import to our notebook.

In [4]:
import pandas as pd
import datetime as dt
import time
import requests

We will first pull information from the NFL reddit and create a pandas dataframe from relevent information gathered.

In [5]:
#pull URL
url = "https://api.pushshift.io/reddit/search/submission?subreddit=nfl"

In [6]:
res = requests.get(url)

In [7]:
#ensure that information was properly pulled
res.status_code

200

In [8]:
assert res.status_code == 200

In [9]:
#see object type we pulled
type(res)

requests.models.Response

In [10]:
#create preliminary dataframe to see all columns we will be pulling
mine = res.json()['data']
df = pd.DataFrame.from_dict(mine)

In [11]:
df.head()

,all_awardings,allow_live_comments,author,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,author_flair_type,author_fullname,...,title,total_awards_received,url,whitelist_status,wls,author_flair_background_color,link_flair_css_class,link_flair_text,crosspost_parent,crosspost_parent_list
0,[],False,Donutman97,chiefs,"[{'e': 'text', 't': 'Chiefs'}]",8ff0a826-9b4a-11e2-885a-12313d164929,Chiefs,dark,richtext,t2_2iy7b629,...,[PETA] THIS is the PETA #SuperBowl ad the @NFL...,0,https://twitter.com/peta/status/12231777582278...,all_ads,6,NaN,NaN,NaN,NaN,NaN
1,[],False,nfl,nfl official,"[{'e': 'text', 't': 'NFL '}, {'a': ':NFL:', 'e...",NaN,NFL :NFL:,dark,richtext,t2_30enjsvu,...,[NFL Research] Mahomes and Jimmy G join Montan...,0,https://twitter.com/NFLResearch/status/1223320...,all_ads,6,,NaN,NaN,NaN,NaN
2,[],False,TinyTornado7,None,[],NaN,None,NaN,text,t2_qs2copj,...,How Much Football Is Even In A Football Broadc...,0,https://fivethirtyeight.com/features/how-much-...,all_ads,6,NaN,NaN,NaN,NaN,NaN
3,[],False,Donutman97,chiefs,"[{'e': 'text', 't': 'Chiefs'}]",8ff0a826-9b4a-11e2-885a-12313d164929,Chiefs,dark,richtext,t2_2iy7b629,...,[PETA] THIS is the PETA #SuperBowl ad the @NFL...,0,https://twitter.com/peta/status/12231777582278...,all_ads,6,NaN,NaN,NaN,NaN,NaN
4,[],False,Zack_scholes,None,[],NaN,None,NaN,text,t2_1mtg7s1h,...,Thought this sourdough alligator in SF was coo...,0,https://i.redd.it/ttf1i5yda6e41.jpg,all_ads,6,NaN,NaN,NaN,NaN,NaN


In [12]:
#print all columns
for i in df.columns:
    print (i)

all_awardings
allow_live_comments
author
author_flair_css_class
author_flair_richtext
author_flair_template_id
author_flair_text
author_flair_text_color
author_flair_type
author_fullname
author_patreon_flair
author_premium
awarders
can_mod_post
contest_mode
created_utc
domain
full_link
gildings
id
is_crosspostable
is_meta
is_original_content
is_reddit_media_domain
is_robot_indexable
is_self
is_video
link_flair_background_color
link_flair_richtext
link_flair_text_color
link_flair_type
locked
media
media_embed
media_only
no_follow
num_comments
num_crossposts
over_18
parent_whitelist_status
permalink
pinned
post_hint
preview
pwls
removed_by_category
retrieved_on
score
secure_media
secure_media_embed
selftext
send_replies
spoiler
stickied
subreddit
subreddit_id
subreddit_subscribers
subreddit_type
thumbnail
thumbnail_height
thumbnail_width
title
total_awards_received
url
whitelist_status
wls
author_flair_background_color
link_flair_css_class
link_flair_text
crosspost_parent
crosspost_paren

In [18]:
#select what columns we are going to keep 
subfields = ['title', 'selftext', 'subreddit', 'created_utc', 'author', 'num_comments', 'score', 'is_self']
df[subfields].head()

,title,selftext,subreddit,created_utc,author,num_comments,score,is_self
0,We Need Signatures,,nba,1580163460,ItsTheDarkBomber,0,1,False
1,Share,,nba,1580163330,Ihousley,0,1,False
2,Kobe made me realize things about certain peop...,[removed],nba,1580163272,Shitzonya,0,1,True
3,I'm a California High School Teacher and today...,,nba,1580163228,HT54,1,1,False
4,Grown men cry!,It's been touching to see the lack of emotiona...,nba,1580163166,kiwibokbok,7,1,True


In [51]:
#credit to Mahdi for help on this function
def query_pushshift(subreddit, kind = 'submission', day_window = 15, n = 15):
    SUBFIELDS = ['title', 'selftext', 'subreddit', 'created_utc', 'author', 'num_comments', 'score', 'is_self']
    # establish base url and stem
    BASE_URL = f"https://api.pushshift.io/reddit/search/{kind}" # also known as the "API endpoint" 
    stem = f"{BASE_URL}?subreddit={subreddit}&size=500" # always pulling max of 500
    # instantiate empty list for temp storage
    posts = []
    # implement for loop with `time.sleep(2)`
    for i in range(1, n + 1):
        URL = "{}&after={}d".format(stem, day_window * i)
        print("Querying from: " + URL)
        response = requests.get(URL)
        assert response.status_code == 200
        mine = response.json()['data']
        df = pd.DataFrame.from_dict(mine)
        posts.append(df)
        time.sleep(2)
    # pd.concat storage list
    full = pd.concat(posts, sort=False)
    # if submission
    if kind == "submission":
        # select desired columns
        full = full[SUBFIELDS]
        # drop duplicates
        full.drop_duplicates(inplace = True)
        # select `is_self` == True
        full = full.loc[full['is_self'] == True]
    # create `timestamp` column
    full['timestamp'] = full["created_utc"].map(dt.date.fromtimestamp)
    print("Query Complete!")    
    return full 


In [52]:
query_pushshift('nfl')

Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=nfl&size=500&after=15d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=nfl&size=500&after=30d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=nfl&size=500&after=45d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=nfl&size=500&after=60d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=nfl&size=500&after=75d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=nfl&size=500&after=90d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=nfl&size=500&after=105d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=nfl&size=500&after=120d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=nfl&size=500&after=135d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=nfl&size=500&after=150d
Querying from: h

,title,selftext,subreddit,created_utc,author,num_comments,score,is_self,timestamp
1,Who is free to help out with a sacko punishment?,Amazon gift card in it for yall haha\n\nwe hav...,nfl,1578889709,spicyboy2727,13,1,True,2020-01-12
4,TIL The President of the NFL Hall of Fame is a...,[removed],nfl,1578889869,No_Cat_No_Cradle,0,1,True,2020-01-12
7,The Browns are like the rich kid who got all t...,,nfl,1578890127,RuneSlayer4421,3,1,True,2020-01-12
8,"14 years later, Norv Turner's prediction of Ji...",During Troy Aikman's Hall of Fame enshrinement...,nfl,1578890138,DraymondShldntWear23,3,1,True,2020-01-12
10,Is Jadeveon Clowney the dirtiest player in the...,After another dirty play from him today and th...,nfl,1578890232,MikeG1990,21,1,True,2020-01-12
...,...,...,...,...,...,...,...,...,...
482,Regarding 2020 NFL Draft,"I was wondering, who are some prospects that a...",nfl,1561031526,dixitsavy,40,30,True,2019-06-20
485,/r/NFL All-Time Draft Pick Tournament: The #28...,**[Introductory Post](https://en.reddit.com/r/...,nfl,1561035134,JaguarGator9,13,82,True,2019-06-20
488,What's the most you would want your team to pa...,Peak off-season shower thought.\n\nI saw a com...,nfl,1561037853,TheDolphinGamer96,10,0,True,2019-06-20
495,Which former player would be the most benefici...,,nfl,1561039216,Walterwhiteboy,18,0,True,2019-06-20


In [55]:
df = query_pushshift('nfl')


Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=nfl&size=500&after=15d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=nfl&size=500&after=30d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=nfl&size=500&after=45d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=nfl&size=500&after=60d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=nfl&size=500&after=75d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=nfl&size=500&after=90d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=nfl&size=500&after=105d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=nfl&size=500&after=120d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=nfl&size=500&after=135d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=nfl&size=500&after=150d
Querying from: h

In [56]:
#see how many rows pulled from the NFL subreddit
df.shape

(2859, 9)

In [70]:
df2 = query_pushshift('football')

Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=football&size=500&after=15d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=football&size=500&after=30d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=football&size=500&after=45d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=football&size=500&after=60d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=football&size=500&after=75d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=football&size=500&after=90d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=football&size=500&after=105d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=football&size=500&after=120d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=football&size=500&after=135d
Querying from: https://api.pushshift.io/reddit/search/submission?subre

In [71]:
# see how many rows pulled from the Football subreddit
df2.shape

(1834, 9)

In [61]:
#merge two reddit pages pulled into a dataframe
all_dfs = [df, df2]

merge_df = pd.concat(all_dfs, sort = False)

In [63]:
merge_df.shape

(4692, 9)

In [22]:
merge_df.head()

,title,selftext,subreddit,created_utc,author,num_comments,score,is_self,timestamp
0,Who is free to help out with a sacko punishment?,Amazon gift card in it for yall haha\n\nwe hav...,nfl,1578889709,spicyboy2727,13,1,True,2020-01-12
3,TIL The President of the NFL Hall of Fame is a...,[removed],nfl,1578889869,No_Cat_No_Cradle,0,1,True,2020-01-12
6,The Browns are like the rich kid who got all t...,NaN,nfl,1578890127,RuneSlayer4421,3,1,True,2020-01-12
7,"14 years later, Norv Turner's prediction of Ji...",During Troy Aikman's Hall of Fame enshrinement...,nfl,1578890138,DraymondShldntWear23,3,1,True,2020-01-12
9,Is Jadeveon Clowney the dirtiest player in the...,After another dirty play from him today and th...,nfl,1578890232,MikeG1990,21,1,True,2020-01-12


In [ ]:
#after evaluating our model within our modeling code notebook, deemed that these were as well unneccessary features
merge_df.drop( columns = ['created_utc', 'score', 'is_self'], inplace = True)

In [27]:
#save dataframe as csv
merge_df.to_csv('scrapped_df.csv')